In [1]:
 import pandas as pd
 import numpy as np

In [2]:
df = pd.read_csv('Train.csv')
y = df.Item_Outlet_Sales
df.drop(columns='Item_Outlet_Sales', inplace=True)
df.replace({'Item_Fat_Content': {'low fat':'Low Fat','LF':'Low Fat', 'reg':'Regular'}}, inplace=True)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1


In [3]:
dfo = df.select_dtypes('object')
dfn = df.select_dtypes(['float64', 'int64'])

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn import set_config
set_config(display="diagram")


numeric_transformers = Pipeline(
    steps = [
            ('imputer', SimpleImputer(strategy="median"))
    ]
)

categorical_transformers = Pipeline(
    steps = [
            ('imputer', SimpleImputer(strategy="most_frequent")), 
             ('encoder', OneHotEncoder())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
                    ('num', numeric_transformers, dfn.columns),
                    ('cat', categorical_transformers, dfo.columns)
    ]
)

In [5]:
regressor = Pipeline(
    steps = [
             ('preprocessor', preprocessor),
             ('regressor', XGBRegressor(objective='reg:squarederror'))
    ]
)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y , test_size=.2, random_state=1)

In [7]:
regressor.fit(df, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Item_Id...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [8]:
regressor.score(df, y)

0.7175524520057209